[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Van-wise/sd-colab/blob/main/vlad/vlad_sd_webui.ipynb)

https://github.com/vladmandic/automatic

https://github.com/Van-wise/sd-colab

**如果遇到报错,请联系up：**

https://space.bilibili.com/1308057/channel/collectiondetail?sid=1365244


In [ ]:
#@title 👇 首次运行！
import sys
import os
import base64
from IPython.display import clear_output

#Check GPU
import tensorflow as tf
print("TensorFlow version:", tf.__version__)
if tf.test.gpu_device_name():
    print("GPU is available")
else:
    print("GPU is NOT available")
    raise Exception("\n没有使用GPU，请在代码执行程序-更改运行时类型-设置为GPU！\n如果不能使用GPU，建议更换账号！")

automatic = '/content/automatic'
w = base64.b64decode(("c2Qtd2VidWk=").encode('ascii')).decode('ascii')

#部署 ubuntu3 环境
%cd /content
!apt -y update -qq
!wget http://launchpadlibrarian.net/367274644/libgoogle-perftools-dev_2.5-2.2ubuntu3_amd64.deb
!wget https://launchpad.net/ubuntu/+source/google-perftools/2.5-2.2ubuntu3/+build/14795286/+files/google-perftools_2.5-2.2ubuntu3_all.deb
!wget https://launchpad.net/ubuntu/+source/google-perftools/2.5-2.2ubuntu3/+build/14795286/+files/libtcmalloc-minimal4_2.5-2.2ubuntu3_amd64.deb
!wget https://launchpad.net/ubuntu/+source/google-perftools/2.5-2.2ubuntu3/+build/14795286/+files/libgoogle-perftools4_2.5-2.2ubuntu3_amd64.deb
!apt install -qq libunwind8-dev
!dpkg -i *.deb
%env LD_PRELOAD=libtcmalloc.so
!rm *.deb
clear_output ()

#部署 GPU 环境
!apt -y install -qq aria2 libcairo2-dev pkg-config python3-dev
!pip install -q torch==2.0.0+cu118 torchvision==0.15.1+cu118 torchaudio==2.0.1+cu118 torchtext==0.15.1 torchdata==0.6.0 --extra-index-url https://download.pytorch.org/whl/cu118 -U
!pip install -q xformers==0.0.19 triton==2.0.0 -U
clear_output ()

#克隆主项目&常用插件
!git clone https://github.com/vladmandic/automatic
!git clone https://huggingface.co/embed/negative {automatic}/models/embeddings
!git clone https://huggingface.co/embed/lora {automatic}/models/Lora/positive
!git clone https://github.com/camenduru/{w}-tunnels {automatic}/extensions/{w}-tunnels
!git clone https://github.com/etherealxx/batchlinks-webui {automatic}/extensions/batchlinks-webui

#姿势&手势插件
!git clone https://github.com/fkunn1326/openpose-editor {automatic}/extensions/openpose-editor
#!git clone https://github.com/hnmr293/posex {automatic}/extensions/posex
#!git clone https://github.com/nonnonstop/{w}-3d-open-pose-editor {automatic}/extensions/{w}-3d-open-pose-editor
#!git clone https://github.com/jexom/{w}-depth-lib {automatic}/extensions/{w}-depth-lib
#!git clone https://github.com/deforum-art/deforum-for-automatic1111-webui {automatic}/extensions/deforum-for-automatic1111-webui #deforum 动画

#安装中文&主题
!rm -r {automatic}/extensions/{w}-bilingual-localization
!git clone https://jihulab.com/vanwise/sdwebui-bilingual-localization-zh_cn {automatic}/extensions/{w}-bilingual-localization
!mkdir -p {automatic}/localizations
!git clone https://github.com/canisminor1990/{w}-kitchen-theme {automatic}/extensions/{w}-kitchen-theme

#同步 config.json (可选)
%cd {automatic}
!rm -r {automatic}/config.json
!rm -r {automatic}/ui-config.json
!wget -O "config.json" "https://jihulab.com/vanwise/{w}-colab/-/raw/main/vlad-{w}/config.json"
!wget -O "ui-config.json" "https://jihulab.com/vanwise/{w}-colab/-/raw/main/vlad-{w}/ui-config.json"

#下载主模型
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/chilloutmix/resolve/main/chilloutmix_NiPrunedFp32Fix.safetensors -d {automatic}/models/Stable-diffusion -o chilloutmix_NiPrunedFp32Fix.safetensors

#vae
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/stabilityai/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.safetensors -d {automatic}/models/VAE -o vae-ft-mse-840000-ema-pruned.safetensors

#Lora
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://civitai.com/api/download/models/16557 -d {automatic}/models/Lora -o ShojoVibev_11.safetensors

#ControlNet
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11e_sd15_ip2p_fp16.safetensors -d {automatic}/extensions-builtin/{w}-controlnet/models -o control_v11e_sd15_ip2p_fp16.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11e_sd15_shuffle_fp16.safetensors -d {automatic}/extensions-builtin/{w}-controlnet/models -o control_v11e_sd15_shuffle_fp16.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15_canny_fp16.safetensors -d {automatic}/extensions-builtin/{w}-controlnet/models -o control_v11p_sd15_canny_fp16.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11f1p_sd15_depth_fp16.safetensors -d {automatic}/extensions-builtin/{w}-controlnet/models -o control_v11f1p_sd15_depth_fp16.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15_inpaint_fp16.safetensors -d {automatic}/extensions-builtin/{w}-controlnet/models -o control_v11p_sd15_inpaint_fp16.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15_lineart_fp16.safetensors -d {automatic}/extensions-builtin/{w}-controlnet/models -o control_v11p_sd15_lineart_fp16.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15_mlsd_fp16.safetensors -d {automatic}/extensions-builtin/{w}-controlnet/models -o control_v11p_sd15_mlsd_fp16.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15_normalbae_fp16.safetensors -d {automatic}/extensions-builtin/{w}-controlnet/models -o control_v11p_sd15_normalbae_fp16.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15_openpose_fp16.safetensors -d {automatic}/extensions-builtin/{w}-controlnet/models -o control_v11p_sd15_openpose_fp16.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15_scribble_fp16.safetensors -d {automatic}/extensions-builtin/{w}-controlnet/models -o control_v11p_sd15_scribble_fp16.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15_seg_fp16.safetensors -d {automatic}/extensions-builtin/{w}-controlnet/models -o control_v11p_sd15_seg_fp16.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15_softedge_fp16.safetensors -d {automatic}/extensions-builtin/{w}-controlnet/models -o control_v11p_sd15_softedge_fp16.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15s2_lineart_anime_fp16.safetensors -d {automatic}/extensions-builtin/{w}-controlnet/models -o control_v11p_sd15s2_lineart_anime_fp16.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11f1e_sd15_tile_fp16.safetensors -d {automatic}/extensions-builtin/{w}-controlnet/models -o control_v11f1e_sd15_tile_fp16.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11e_sd15_ip2p_fp16.yaml -d {automatic}/extensions-builtin/{w}-controlnet/models -o control_v11e_sd15_ip2p_fp16.yaml
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11e_sd15_shuffle_fp16.yaml -d {automatic}/extensions-builtin/{w}-controlnet/models -o control_v11e_sd15_shuffle_fp16.yaml
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11p_sd15_canny_fp16.yaml -d {automatic}/extensions-builtin/{w}-controlnet/models -o control_v11p_sd15_canny_fp16.yaml
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11f1p_sd15_depth_fp16.yaml -d {automatic}/extensions-builtin/{w}-controlnet/models -o control_v11f1p_sd15_depth_fp16.yaml
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11p_sd15_inpaint_fp16.yaml -d {automatic}/extensions-builtin/{w}-controlnet/models -o control_v11p_sd15_inpaint_fp16.yaml
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11p_sd15_lineart_fp16.yaml -d {automatic}/extensions-builtin/{w}-controlnet/models -o control_v11p_sd15_lineart_fp16.yaml
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11p_sd15_mlsd_fp16.yaml -d {automatic}/extensions-builtin/{w}-controlnet/models -o control_v11p_sd15_mlsd_fp16.yaml
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11p_sd15_normalbae_fp16.yaml -d {automatic}/extensions-builtin/{w}-controlnet/models -o control_v11p_sd15_normalbae_fp16.yaml
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11p_sd15_openpose_fp16.yaml -d {automatic}/extensions-builtin/{w}-controlnet/models -o control_v11p_sd15_openpose_fp16.yaml
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11p_sd15_scribble_fp16.yaml -d {automatic}/extensions-builtin/{w}-controlnet/models -o control_v11p_sd15_scribble_fp16.yaml
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11p_sd15_seg_fp16.yaml -d {automatic}/extensions-builtin/{w}-controlnet/models -o control_v11p_sd15_seg_fp16.yaml
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11p_sd15_softedge_fp16.yaml -d {automatic}/extensions-builtin/{w}-controlnet/models -o control_v11p_sd15_softedge_fp16.yaml
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11p_sd15s2_lineart_anime_fp16.yaml -d {automatic}/extensions-builtin/{w}-controlnet/models -o control_v11p_sd15s2_lineart_anime_fp16.yaml
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11f1e_sd15_tile_fp16.yaml -d {automatic}/extensions-builtin/{w}-controlnet/models -o control_v11f1e_sd15_tile_fp16.yaml
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/t2iadapter_style_sd14v1.pth -d {automatic}/extensions-builtin/{w}-controlnet/models -o t2iadapter_style_sd14v1.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/t2iadapter_sketch_sd14v1.pth -d {automatic}/extensions-builtin/{w}-controlnet/models -o t2iadapter_sketch_sd14v1.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/t2iadapter_seg_sd14v1.pth -d {automatic}/extensions-builtin/{w}-controlnet/models -o t2iadapter_seg_sd14v1.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/t2iadapter_openpose_sd14v1.pth -d {automatic}/extensions-builtin/{w}-controlnet/models -o t2iadapter_openpose_sd14v1.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/t2iadapter_keypose_sd14v1.pth -d {automatic}/extensions-builtin/{w}-controlnet/models -o t2iadapter_keypose_sd14v1.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/t2iadapter_depth_sd14v1.pth -d {automatic}/extensions-builtin/{w}-controlnet/models -o t2iadapter_depth_sd14v1.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/t2iadapter_color_sd14v1.pth -d {automatic}/extensions-builtin/{w}-controlnet/models -o t2iadapter_color_sd14v1.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/t2iadapter_canny_sd14v1.pth -d {automatic}/extensions-builtin/{w}-controlnet/models -o t2iadapter_canny_sd14v1.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/t2iadapter_canny_sd15v2.pth -d {automatic}/extensions-builtin/{w}-controlnet/models -o t2iadapter_canny_sd15v2.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/t2iadapter_depth_sd15v2.pth -d {automatic}/extensions-builtin/{w}-controlnet/models -o t2iadapter_depth_sd15v2.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/t2iadapter_sketch_sd15v2.pth -d {automatic}/extensions-builtin/{w}-controlnet/models -o t2iadapter_sketch_sd15v2.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/t2iadapter_zoedepth_sd15v1.pth -d {automatic}/extensions-builtin/{w}-controlnet/models -o t2iadapter_zoedepth_sd15v1.pth
clear_output ()

#安装requirements
!pip install -r {automatic}/requirements.txt
!pip install pycloudflared
clear_output ()

#运行installer.py
!python installer.py
clear_output ()

#启动
%cd {automatic}
!python launch.py --share --listen --no-download --theme=dark --disable-safe-unpickle --skip-update

In [ ]:
#@title 👇启动 SDwebui
%cd {automatic}
!python launch.py --share --listen --no-download --theme=dark --disable-safe-unpickle --skip-update

In [ ]:
#@title 链接谷歌硬盘！
import sys
import os
import base64
import importlib.util
from IPython.display import clear_output

sdw = base64.b64decode(("c3RhYmxlLWRpZmZ1c2lvbi13ZWJ1aQ==").encode('ascii')).decode('ascii')
w = base64.b64decode(("c2Qtd2VidWk=").encode('ascii')).decode('ascii')
gwebui_dir = f'/content/drive/MyDrive/{sdw}'
automatic = '/content/automatic'

#装载 GoogleDrive
from google.colab import drive
drive.mount('/content/drive')

#链接outputs
!mkdir -p {gwebui_dir}/outputs
!test -d {automatic}/outputs && test -d {gwebui_dir}/outputs && ln -sf {gwebui_dir}/outputs {automatic}/outputs

#链接 models
!mkdir -p {gwebui_dir}/models/Stable-diffusion
!test -d {automatic}/models/Stable-diffusion && test -d {gwebui_dir}/models/Stable-diffusion && ln -sf {gwebui_dir}/models/Stable-diffusion {automatic}/models/Stable-diffusion

#链接 Lora
!mkdir -p {gwebui_dir}/extensions/{w}-additional-networks/models/lora
!test -d {automatic}/extensions/{w}-additional-networks/models/lora && test -d {gwebui_dir}/extensions/{w}-additional-networks/models/lora && ln -sf {gwebui_dir}/extensions/{w}-additional-networks/models/lora {automatic}/extensions/{w}-additional-networks/models/lora

!mkdir -p {gwebui_dir}/models/Lora
!test -d {automatic}/models/Lora && test -d {gwebui_dir}/models/Lora && ln -sf {gwebui_dir}/models/Lora {automatic}/models/Lora

#链接 VAE
!mkdir -p {gwebui_dir}/models/VAE
!test -d {automatic}/models/VAE && test -d {gwebui_dir}/models/VAE && ln -sf {gwebui_dir}/models/VAE {automatic}/models/VAE

#链接 embeddings
!mkdir -p {gwebui_dir}/embeddings
!test -d {automatic}/models/embeddings && test -d {gwebui_dir}/embeddings && ln -sf {gwebui_dir}/embeddings {automatic}/models/embeddings

In [ ]:
# 重新链接 models
# 一般需要要运行
!test -d {gwebui_dir}/models/Stable-diffusion && (test -z "$(ls -A {gwebui_dir}/models/Stable-diffusion)" || ln -sf {gwebui_dir}/models/Stable-diffusion/* {automatic}/models/Stable-diffusion/)
!test -d {gwebui_dir}/extensions/{w}-additional-networks/models/lora && (test -z "$(ls -A {gwebui_dir}/extensions/{w}-additional-networks/models/lora" || ln -sf {gwebui_dir}/extensions/{w}-additional-networks/models/lora* {automatic}/extensions/{w}-additional-networks/models/lora)
!test -d {gwebui_dir}/models/Lora && (test -z "$(ls -A {gwebui_dir}/models/Lora)" || ln -sf {gwebui_dir}/models/Lora/* {automatic}/models/Lora/)
!test -d {gwebui_dir}/models/VAE && (test -z "$(ls -A {gwebui_dir}/models/VAE)" || ln -sf {gwebui_dir}/models/VAE* {automatic}/models/VAE)
!test -d {gwebui_dir}/embeddings && (test -z "$(ls -A {gwebui_dir}/embeddings)" || ln -sf {gwebui_dir}/embeddings* {automatic}/models/embeddings)
!test -d {gwebui_dir}/outputs && (test -z "$(ls -A {gwebui_dir}/outputs)" || ln -sf {gwebui_dir}/outputs* {automatic}/outputs)

# **以下是下载操作：**




In [ ]:
#下载模型到 SDwebui （colab）
#下载主模型，请根据自己的需求下载，修改下载地址和文件名即可
!wget https://civitai.com/api/download/models/11745 -O {automatic}/models/Stable-diffusion/Chilloutmix-Ni-pruned-fp32-fix.safetensors

#下载 Lora 模型
!wget https://civitai.com/api/download/models/16557 -O {automatic}/models/Lora/ShojoVibev_11.safetensors
#!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://civitai.com/api/download/models/16557 -d {automatic}/models/Lora -o ShojoVibev_11.safetensors

In [ ]:
#下载模型到 GoogleDrive
#下载链接，文件名，models路径
url = "https://civitai.com/api/download/models/16557" #models文件下载链接
filename = "ShojoVibev_11.safetensors" #保存文件名 .safetensors or .pt
save_dir = f"{gwebui_dir}/models/Lora"  # f"{gwebui_dir}/models/Lora/Stable-diffusion"

!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M -o {filename} -d {save_dir} {url}

# **以下是备份操作：**

In [ ]:
# 备份 config.json
!test -f {automatic}/config.json && cp {automatic}/config.json {gwebui_dir}/config.json
!test -f {automatic}/ui-config.json && cp {automatic}/ui-config.json {gwebui_dir}/ui-config.json

In [ ]:
# 使用云盘 config.json
!test -d {gwebui_dir}/config.json && test -d {automatic}/config.json && rm -rf {automatic}/config.json
!test -f {automatic}/config.json && cp {gwebui_dir}/config.json {automatic}/config.json

In [ ]:
# 备份 outputs 到 drive ，并链接。
!test -d {gwebui_dir}/outputs && cp -r {automatic}/outputs {gwebui_dir}
!test -d {gwebui_dir}/outputs && test -d {automatic}/outputs && rm -rf {automatic}/outputs
!ln -s {gwebui_dir}/outputs {automatic}/outputs

In [ ]:
#备份 Lora 
# w_dir:colab文件目录；g_dir:云盘文件目录
import os
import shutil

w_dir = f"{automatic}/models/Lora" #f"{automatic}/models/Stable-diffusion"
g_dir = f"{gwebui_dir}/models/Lora" #f"{gwebui_dir}/models/Stable-diffusion"

for filename in os.listdir(w_dir):
    w_path = os.path.join(w_dir, filename)
    g_path = os.path.join(g_dir, filename)

    if os.path.isfile(w_path) and not os.path.exists(g_path):
        shutil.copyfile(w_path, g_path)

In [ ]:
#备份 VAE
import os
import shutil

w_dir = f"{automatic}/models/VAE"
g_dir = f"{gwebui_dir}/models/VAE"

for filename in os.listdir(w_dir):
    w_path = os.path.join(w_dir, filename)
    g_path = os.path.join(g_dir, filename)

    if os.path.isfile(w_path) and not os.path.exists(g_path):
        shutil.copyfile(w_path, g_path)

In [ ]:
#备份 models
#主模型占容量较大，不建议备份。
import os
import shutil

w_dir = f"{automatic}/models/Stable-diffusion"
g_dir = f"{gwebui_dir}/models/Stable-diffusion"

for filename in os.listdir(w_dir):
    w_path = os.path.join(w_dir, filename)
    g_path = os.path.join(g_dir, filename)

    if os.path.isfile(w_path) and not os.path.exists(g_path):
        shutil.copyfile(w_path, g_path)